In [13]:
import pandas as pd
import numpy as np
import pickle
import re
import timeit
import spacy

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import LdaMulticore, ldaseqmodel
from nltk.corpus import stopwords
from gensim.matutils import hellinger
import helper as he
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
# # loading our corpus and dictionary
# try:
#     dictionary = Dictionary.load('datasets/news_dictionary')
# except FileNotFoundError as e:
#     raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")
# corpus = bleicorpus.BleiCorpus('datasets/news_corpus')
# # it's very important that your corpus is saved in order of your time-slices!

# time_slice = [438, 430, 456]

In [18]:
with open('../data/preprocessed_data/aadhar_corp.pkl','rb') as f:
    data_lemmatized, id2word, corpus = pickle.load(f)
    
with open('../data/massmedia-data/aadhar_date_indexed.pkl','rb') as f:
    _, time_index_arr = pickle.load(f)

In [38]:
id2word = Dictionary(data_lemmatized)
id2word.filter_extremes(no_below=5, no_above=0.1, keep_n=100, keep_tokens=None)

corpus = [id2word.doc2bow(text) for text in data_lemmatized]

2019-12-04 20:14:53,878 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-12-04 20:14:57,865 : INFO : adding document #10000 to Dictionary(53262 unique tokens: ['aadhaar', 'access', 'adilabad', 'also', 'anand']...)
2019-12-04 20:15:00,046 : INFO : built Dictionary(65957 unique tokens: ['aadhaar', 'access', 'adilabad', 'also', 'anand']...) from 13908 documents (total 2792599 corpus positions)
2019-12-04 20:15:00,246 : INFO : discarding 65857 tokens: [('aadhaar', 11563), ('adilabad', 44), ('also', 8094), ('anand', 119), ('anukunta', 1), ('attend', 554), ('basis', 1004), ('become', 1791), ('bring', 2087), ('card', 8578)]...
2019-12-04 20:15:00,247 : INFO : keeping 100 tokens which were in no less than 5 and no more than 1390 (=10.0%) documents
2019-12-04 20:15:00,304 : INFO : resulting dictionary: Dictionary(100 unique tokens: ['access', 'holder', 'other', 'special', 'village']...)


In [39]:
def generate_timeslice_data(index_arr):
    arr = []
    for i in index_arr.values():
        arr.append(i)
        
    diff_arr=[]
    for i in range(1,len(arr)):
        diff_arr.append(arr[i]-arr[i-1])
    return diff_arr

In [40]:
time_slices = generate_timeslice_data(time_index_arr)

In [41]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=id2word, time_slice=time_slices, num_topics=35, passes=1)

2019-12-04 20:15:02,249 : INFO : using symmetric eta at 0.02857142857142857
2019-12-04 20:15:02,251 : INFO : using serial LDA version on this node
2019-12-04 20:15:02,258 : INFO : running online (single-pass) LDA training, 35 topics, 1 passes over the supplied corpus of 13908 documents, updating model once every 2000 documents, evaluating perplexity every 13908 documents, iterating 50x with a convergence threshold of 0.001000
2019-12-04 20:15:02,260 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-12-04 20:15:02,266 : INFO : PROGRESS: pass 0, at document #2000/13908
2019-12-04 20:15:03,635 : INFO : merging changes from 2000 documents into a model of 13908 documents
2019-12-04 20:15:03,652 : INFO : topic #23 (0.010): 0.064*"consumer" + 0.031*"meet" + 0.031*"apply" + 0.030*"customer" + 0.027*"clear" + 0.026*"june" + 0.022*"meeting" + 0.022*"total" + 0.021*"election" + 0.021*"education"
2019-12-04 20:

IndexError: index 394 is out of bounds for axis 1 with size 394

In [ ]:
print(ldaseq)